# Federated Learning with Diff Privacy using PySyft
# Server Side

When effectively implementing Federated Learning in real world applications, it is important to build an architeture where data in a remote server could be accessed in order to train and test the model. 

In the previous examples this is implemented using virtual clients. This example attempts to implement it over websockets.

This notebook is the server side where the model is defined.

In [1]:
import syft as sy

hook = sy.TorchHook()

In [2]:
hook.local_worker = sy.SocketWorker(hook=hook,
                            id=3,
                            port=3000,
                            is_pointer=False,
                            is_client_worker=False,
                            verbose=True)
sy.local_worker = hook.local_worker

Starting Socket Worker...
Ready to receive commands...


#### Now data is defined on the user side

In [3]:
from __future__ import print_function
import argparse
import numpy as np
from numpy.random import randint
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import TensorDataset, DataLoader

print(torch.__version__)
print(torch.cuda.is_available())
import pickle

f = open('../other/data/boston_housing.pickle','rb')
((X, y), (X_test, y_test)) = pickle.load(f)
f.close()

X = torch.from_numpy(X).type(torch.FloatTensor)
y = torch.from_numpy(y).type(torch.FloatTensor)
X_test = torch.from_numpy(X_test).type(torch.FloatTensor)
y_test = torch.from_numpy(y_test).type(torch.FloatTensor)
# preprocessing
mean = X.mean(0, keepdim=True)
dev = X.std(0, keepdim=True)
mean[:, 3] = 0. # the feature at column 3 is binary,
dev[:, 3] = 1.  # so I'd rather not standardize it
X = (X - mean) / dev
X_test = (X_test - mean) / dev
# X_train = TensorDataset(X, y)
train = TensorDataset(X, y)
test = TensorDataset(X_test, y_test)
train_loader = DataLoader(train, batch_size=8, shuffle=True)
test_loader = DataLoader(test, batch_size=8, shuffle=True)

0.3.1.post2
False


In [4]:
train_dataset = []

for batch_idx, (data,target) in enumerate(train_loader):
    data = Variable(data)
    target = Variable(target.float())
    train_dataset.append((data, target))

# for entry_idx, (data,target) in enumerate(train):
#     data = Variable(data)
#     target = Variable(target)
#     train_dataset.append((data, target))

In [ ]:
for entry in range(len(train_dataset)):
    train_dataset[entry][0].set_id('#X' + str([entry][0]))
    train_dataset[entry][1].set_id('#y' + str([entry][0]))
    
    

In [ ]:
hook.local_worker.listen()